In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler , Normalizer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import train_test_split
from tensorflow import keras


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# import data 
dataset = pd.read_csv('../input/diabetes/diabetes.csv')
dataset.head()

In [ ]:
!ls -ltrha

In [ ]:
dataset.info()

In [ ]:
dataset.isnull().sum()

# It's time to begin

In [ ]:
X = dataset.copy()
y = X.pop('Outcome')

## ------ preprocessing ------ ##

preprocessor = make_column_transformer(
    (StandardScaler(),
     make_column_selector(dtype_include=np.number)),
)

preprocessor.fit(X)
X = preprocessor.transform(X)

## ------ preprocessing ------ ##

In [ ]:
type(dataset.columns)

In [ ]:
dataset.head()

In [ ]:
scaled_dataset = pd.DataFrame(X,columns=dataset.columns[:-1])
scaled_dataset.head()

In [ ]:
dataset.plot()

In [ ]:
scaled_dataset.plot()

In [ ]:
train_x, test_x ,train_y , test_y = train_test_split(
    X,y, test_size=0.33, random_state=1)

In [ ]:
model = keras.Sequential([
 
    keras.layers.Input((8,)),
#     keras.layers.Dropout(0.3),
    keras.layers.Dense(150 ,activation='relu', kernel_regularizer = keras.regularizers.L2()),
#     keras.layers.Dropout(0.3),
    keras.layers.Dense(75 ,activation='relu', kernel_regularizer = keras.regularizers.L2(),),
    keras.layers.Dense(1 ,activation='sigmoid'),
  
])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss= keras.losses.BinaryCrossentropy(),
    metrics = ['accuracy'],
    )

history = model.fit(
    x=train_x,
    y=train_y,
    validation_data=(test_x,test_y),
    batch_size=28,
    epochs=30
)

In [ ]:
history_df = pd.DataFrame(history.history)
# Start the plot at epoch 5. You can change this to get a different view.
history_df.loc[:, ['loss','val_loss']].plot();
history_df.loc[:, ['accuracy','val_accuracy']].plot();

In [ ]:
model.evaluate(test_x,test_y)

In [ ]:
y__predict=model.predict(test_x)
y__real = test_y

y__predict = [ 1 if y>=0.5 else 0 for y in y__predict]

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
cm = confusion_matrix(y__predict, y__real)
from mlxtend.plotting import plot_confusion_matrix
fig, ax = plot_confusion_matrix(conf_mat=cm)
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
print('Confusion Matrix :')
print(cm) 
print('Accuracy Score :',accuracy_score(y__predict, y__real))
print('Report : ')
print(classification_report(y__predict, y__real))